### training 

In [ ]:
import os
import cv2
import xml.etree.ElementTree as ET
import random
from shutil import copyfile
from ultralytics import YOLO

In [ ]:
model_2 = YOLO('yolo11s.pt')

In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

In [ ]:
from ultralytics import YOLO

# Load model
model_2 = YOLO("yolo11s.pt")

# Train on GPU (device 0)
model_2.train(
    data=r"pth_to_data_yaml_file\data.yaml",
    epochs=100,
    batch=16,
    imgsz=640,
    name="YOLO_Model",
    device=0,        # <-- FORCE GPU (0 = first GPU)
    workers=8,       # data loading speed (try 4-16)
    amp=True         # mixed precision (faster on modern GPUs)
)


### testing

In [ ]:
import os
import glob
from collections import defaultdict
import numpy as np

from ultralytics import YOLO

In [12]:
# Paper-style thresholds
IOU_TH = 0.45
CONF_TH = 0.10


In [13]:
def yolo_txt_to_xyxy(txt_path, img_w, img_h):
    boxes = []
    if not os.path.exists(txt_path):
        return boxes

    with open(txt_path, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 5:
                continue

            cls = int(float(parts[0]))
            xc, yc, w, h = map(float, parts[1:5])

            x1 = (xc - w / 2) * img_w
            y1 = (yc - h / 2) * img_h
            x2 = (xc + w / 2) * img_w
            y2 = (yc + h / 2) * img_h

            boxes.append((cls, x1, y1, x2, y2))
    return boxes


In [14]:
def box_iou_xyxy(a, b):
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b

    inter_x1 = max(ax1, bx1)
    inter_y1 = max(ay1, by1)
    inter_x2 = min(ax2, bx2)
    inter_y2 = min(ay2, by2)

    inter_w = max(0.0, inter_x2 - inter_x1)
    inter_h = max(0.0, inter_y2 - inter_y1)
    inter = inter_w * inter_h

    area_a = max(0.0, ax2 - ax1) * max(0.0, ay2 - ay1)
    area_b = max(0.0, bx2 - bx1) * max(0.0, by2 - by1)

    union = area_a + area_b - inter + 1e-9
    return inter / union


In [15]:
def evaluate_image(preds, gts, iou_th=IOU_TH, conf_th=CONF_TH):
    gt_correct = [False] * len(gts)
    pred_used = [False] * len(preds)

    for gi, gt in enumerate(gts):
        gt_cls, gx1, gy1, gx2, gy2 = gt
        best_iou = 0.0
        best_pi = -1

        for pi, pr in enumerate(preds):
            pr_cls, pr_conf, px1, py1, px2, py2 = pr
            if pr_conf < conf_th or pr_cls != gt_cls or pred_used[pi]:
                continue

            iou = box_iou_xyxy(
                (px1, py1, px2, py2),
                (gx1, gy1, gx2, gy2)
            )

            if iou > best_iou:
                best_iou = iou
                best_pi = pi

        if best_pi != -1 and best_iou >= iou_th:
            gt_correct[gi] = True
            pred_used[best_pi] = True

    return gt_correct, pred_used


In [16]:
MODEL_PATH = r"D:\Submitted Matrial (conference&journal)\Sagittal Data Artical\V0.47 Dataset analysis\dataspitting\runs\detect\YOLO_Model4\weights\best.pt"

IMG_DIR = r"D:\Submitted Matrial (conference&journal)\Sagittal Data Artical\V0.47 Dataset analysis\dataspitting\DatasetV0.47_YOLO\images\val"

LBL_DIR = r"D:\Submitted Matrial (conference&journal)\Sagittal Data Artical\V0.47 Dataset analysis\dataspitting\DatasetV0.47_YOLO\labels\val"


In [17]:
model = YOLO(MODEL_PATH)
names = model.names if hasattr(model, "names") else None

img_paths = []
for ext in ["*.png", "*.jpg", "*.jpeg", "*.bmp"]:
    img_paths.extend(glob.glob(os.path.join(IMG_DIR, ext)))

img_paths = sorted(list(set(img_paths)))
len(img_paths)


272

In [18]:
gt_total = defaultdict(int)
gt_correct = defaultdict(int)

TP = defaultdict(int)
FP = defaultdict(int)
FN = defaultdict(int)

for img_path in img_paths:
    result = model.predict(
        source=img_path,
        conf=CONF_TH,
        iou=0.7,
        verbose=False
    )[0]

    img_h, img_w = result.orig_shape

    base = os.path.splitext(os.path.basename(img_path))[0]
    gt_path = os.path.join(LBL_DIR, base + ".txt")
    gts = yolo_txt_to_xyxy(gt_path, img_w, img_h)

    preds = []
    if result.boxes is not None and len(result.boxes) > 0:
        b = result.boxes
        for xyxy, conf, cls in zip(
            b.xyxy.cpu().numpy(),
            b.conf.cpu().numpy(),
            b.cls.cpu().numpy().astype(int)
        ):
            preds.append((cls, conf, *xyxy))

    gt_flags, pred_flags = evaluate_image(preds, gts)

    for i, gt in enumerate(gts):
        c = gt[0]
        gt_total[c] += 1
        if gt_flags[i]:
            gt_correct[c] += 1
            TP[c] += 1
        else:
            FN[c] += 1

    for i, pr in enumerate(preds):
        if not pred_flags[i]:
            FP[pr[0]] += 1


In [19]:
print("Class | GT | TrueDet% | Precision | Recall | F1")
print("-" * 55)

precisions = []
recalls = []
f1s = []
weights = []

for c in sorted(gt_total.keys()):
    gt = gt_total[c]
    tp, fp, fn = TP[c], FP[c], FN[c]

    true_det = (gt_correct[c] / gt) * 100 if gt > 0 else 0
    precision = tp / (tp + fp) * 100 if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) * 100 if (tp + fn) > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    cname = names[c] if names else str(c)
    print(f"{cname:5s} | {gt:3d} | {true_det:8.2f} | {precision:9.2f} | {recall:6.2f} | {f1:6.2f}")

    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)
    weights.append(gt)


Class | GT | TrueDet% | Precision | Recall | F1
-------------------------------------------------------
L1-L2 | 100 |    93.00 |     64.14 |  93.00 |  75.92
L2-L3 | 142 |    92.96 |     80.49 |  92.96 |  86.27
L3-L4 | 152 |    96.71 |     79.03 |  96.71 |  86.98
L4-L5 | 116 |    92.24 |     72.30 |  92.24 |  81.06
L5-S1 |  46 |    82.61 |     50.67 |  82.61 |  62.81


In [20]:
total_gt = sum(gt_total.values())
overall_true = sum(gt_correct.values()) / total_gt * 100

macro_p = np.mean(precisions)
macro_r = np.mean(recalls)
macro_f1 = np.mean(f1s)

weighted_p = np.average(precisions, weights=weights)
weighted_r = np.average(recalls, weights=weights)
weighted_f1 = np.average(f1s, weights=weights)

print("\n=== SUMMARY ===")
print(f"Overall True Detection (%): {overall_true:.2f}")
print(f"Macro    P/R/F1 (%): {macro_p:.2f} / {macro_r:.2f} / {macro_f1:.2f}")
print(f"Weighted P/R/F1 (%): {weighted_p:.2f} / {weighted_r:.2f} / {weighted_f1:.2f}")



=== SUMMARY ===
Overall True Detection (%): 92.99
Macro    P/R/F1 (%): 69.32 / 91.50 / 78.61
Weighted P/R/F1 (%): 72.97 / 92.99 / 81.58
